In [10]:
import torch.nn as nn
import torch
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
#import torchtext
import io
from happytransformer import HappyTextClassification 


In [11]:
!nvidia-smi
print(torch.cuda.is_available())

Sun Jan 15 03:30:33 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 512.77       Driver Version: 512.77       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
| 18%   39C    P8    18W / 215W |   5287MiB /  8192MiB |     25%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [12]:
happy_tc = HappyTextClassification("DISTILBERT", "distilbert-base-uncased-finetuned-sst-2-english")

loading configuration file config.json from cache at C:\Users\Melina/.cache\huggingface\hub\models--distilbert-base-uncased-finetuned-sst-2-english\snapshots\bfdd146ea2b6807255b73527f1327ca12b6ed5c4\config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased-finetuned-sst-2-english",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "finetuning_task": "sst-2",
  "hidden_dim": 3072,
  "id2label": {
    "0": "NEGATIVE",
    "1": "POSITIVE"
  },
  "initializer_range": 0.02,
  "label2id": {
    "NEGATIVE": 0,
    "POSITIVE": 1
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.25.1",
  "vocab_size": 30522
}

loading weights file py

In [13]:
happy_tc = HappyTextClassification("DISTILBERT", "distilbert-base-uncased",num_labels=3)
from happytransformer import TCTrainArgs
args = TCTrainArgs(batch_size=8)
happy_tc.train("train.csv", args=args)


loading configuration file config.json from cache at C:\Users\Melina/.cache\huggingface\hub\models--distilbert-base-uncased\snapshots\1c4513b2eedbda136f57676a34eea67aba266e5c\config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.25.1",
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at C:\Users\Melina/.cache\huggingface\hub\models--dist

  0%|          | 0/5625 [00:00<?, ?it/s]

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 0.6234, 'learning_rate': 4.555555555555556e-05, 'epoch': 0.27}
{'loss': 0.5044, 'learning_rate': 4.111111111111111e-05, 'epoch': 0.53}
{'loss': 0.4704, 'learning_rate': 3.6666666666666666e-05, 'epoch': 0.8}
{'loss': 0.4272, 'learning_rate': 3.222222222222223e-05, 'epoch': 1.07}


In [9]:
print(happy_tc.classify_text("I am so happy about this!").label[-1])
print(happy_tc.classify_text("This is the most dissapointing news I have had in a while."))
print(happy_tc.classify_text("This house was built three years ago."))
print(happy_tc.classify_text("I am amazed by how amazing these nuts are!"))
print(happy_tc.classify_text("They have no milk, this sucks."))

c:\Users\Melina\anaconda3\lib\site-packages\transformers\pipelines\base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


2
TextClassificationResult(label='LABEL_2', score=0.9996908903121948)
TextClassificationResult(label='LABEL_2', score=0.99887615442276)
TextClassificationResult(label='LABEL_2', score=0.9997308850288391)
TextClassificationResult(label='LABEL_0', score=0.988473117351532)


In [6]:
import evaluate
accuracy_metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")

In [ ]:
predictions = []
with open('src/test.csv') as csv_file:
            csv_reader = csv.reader(csv_file, delimiter='\n')
            for row in csv_reader:
                predictions.append(happy_tc.classify_text(row).label[-1])

test_labels = [2]*5000+[1]*5000+[0]*5000

In [ ]:
accuracy_output = accuracy_metric.compute(references=test_labels, predictions=predictions)
print(accuracy_output)
precision__output = precision_metric.compute(references=test_labels, predictions=predictions)
print(precision__output)
recall_output = recall_metric.compute(references=test_labels, predictions=predictions)
print(recall_output)

In [14]:
#model = fasttext.train_unsupervised('data/enwik9')
def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = map(float, tokens[1:])
    return data



In [9]:
class sentiment_LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_class):
        super(TweetRNN, self).__init__()
        self.emb = nn.Embedding.from_pretrained(glove.vectors) # Replace with embeddings
        self.hidden_size = hidden_size
        self.rnn = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_class)

    def forward(self, x): 
        # Look-up the embeddings 
        x = self.emb(x) 
        # Set the initial hidden states 
        h0 = torch.zeros(1, x.size(0), self.hidden_size) 
        c0 = torch.zeros(1, x.size(0), self.hidden_size) 
        # Forward propagate the RNN 
        out, __ = self.rnn(x, (h0, c0)) 
        # Pass the output of the last step to the classifier return 
        self.fc(out[:,-1,:])

